# Evaluation Pipeline

The evaluation system is divided into **three main stages**:

| Stage | Purpose | Core Functions |
|--------|----------|----------------|
| **A. Inference / Test Phase** | Generate model predictions for each prompt type and save JSON results | `test_binary_prompts()` / `test_multilabel_prompts()` |
| **B. Processing & Comparison Phase** | Read JSON results, merge with ground truth, compute metrics, and compare prompts | `process_results_file()`, `_binary_metrics()`, `_multilabel_metrics()`, `compare_prompts()` |
| **C. Final Evaluation & Visualization Phase** | Evaluate best-performing prompts in depth and visualize results | `final_evaluation_binary()`, `final_evaluation_multilabel()`, `plot_prompt_comparison()`, `run_prompt_comparison_and_final()` |

Each stage produces intermediate outputs (JSONs, metrics, plots) that can be reused without recomputation.


## 3. Stage B — Processing & Comparison Phase
This phase processes saved results and computes metrics per prompt.
`process_results_file(file_path, df_truth, task)`
Reads one JSON results file and merges it with ground truth.
Duties:
* Load JSON (binary_*.json or multilabel_*.json).
* Merge with ground-truth DataFrame on id.
* Convert into a clean numeric format.

In [1]:
# ============================================================
# EVALUATION PIPELINE: Prompt Comparison + Final Evaluation
# ============================================================
from __future__ import annotations
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import json
import re
import os
import math
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image


# ---------- Utilities ---------------------------------------------------------
def _ensure_list(x):
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return []
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        # split comma-joined strings like "Emotional, Physical"
        parts = [p.strip() for p in re.split(r"[;,]", x) if p.strip()]
        return parts if parts else [x]
    return [x]

def _try_extract_scores(record: dict) -> Optional[Dict[str, float]]:
    """
    Try to extract per-class probabilities/scores if your model emitted them
    inside raw_response JSON (e.g., {"probs": {"IPV":0.81,"NOT_IPV":0.19}} or
    {"scores":{"Physical":0.4,...}}). If not found, return None.
    """
    raw = record.get("raw_response", "")
    # Try to locate JSON block between <json>...</json> first
    m = re.search(r"<json[^>]*>\s*(\{.*?\})\s*</json>", raw, re.DOTALL | re.IGNORECASE)
    text_blocks = []
    if m:
        text_blocks.append(m.group(1))
    else:
        # fallback: any JSON-like object
        m2 = re.search(r"(\{.*\})", raw, re.DOTALL)
        if m2:
            text_blocks.append(m2.group(1))

    for block in text_blocks:
        try:
            parsed = json.loads(block)
            # Common keys people use
            for key in ("probs", "probabilities", "scores", "confidences"):
                if key in parsed and isinstance(parsed[key], dict):
                    # normalize keys to str, vals to float
                    return {str(k): float(v) for k, v in parsed[key].items()}
        except Exception:
            continue
    return None

def _binarize_labels(labels: List[str], positive="IPV") -> int:
    """Return 1 if positive present, else 0."""
    s = {lbl.strip() for lbl in _ensure_list(labels)}
    return 1 if positive in s else 0

def _one_vs_rest_columns() -> List[str]:
    return ["Physical", "Emotional", "Sexual"]

# ---------- Helper: process a results file ------------------------------------

def process_results_file(
    file_path: str | Path,
    df_truth: pd.DataFrame,
    task: str,  # "binary" or "multilabel"
    cache_dir: str | Path = "../1_LLM_Eval/eval_cache"
) -> Dict:
    """
    Load one result JSON file, merge with ground truth, and produce a tidy frame:
    Returns dict with:
      - 'file': file_path
      - 'prompt_type': inferred from file name
      - 'task': task
      - 'df': DataFrame with columns:
          id, y_true, y_pred, y_score (binary)
          ...or for multilabel: id, and for each class C in {Physical, Emotional, Sexual}:
              y_true_C, y_pred_C, y_score_C (if available)
      - 'has_scores': whether per-sample probabilities were found
    Caching: saves a compact .parquet keyed by mtime so future calls reuse.
    """
    file_path = Path(file_path)
    cache_dir = Path(cache_dir)
    cache_dir.mkdir(parents=True, exist_ok=True)
    stamp = f"{file_path.stat().st_mtime_ns}"
    cache_key = cache_dir / f"{file_path.stem}_{stamp}.parquet"

    if cache_key.exists():
        out = pd.read_parquet(cache_key)
        # Rehydrate metadata saved in footer
        meta_path = cache_dir / f"{file_path.stem}_{stamp}.meta.json"
        meta = json.loads(Path(meta_path).read_text()) if meta_path.exists() else {}
        return {
            "file": str(file_path),
            "prompt_type": meta.get("prompt_type", file_path.stem),
            "task": task,
            "df": out.copy(),
            "has_scores": bool(meta.get("has_scores", False)),
        }

    # Read results JSON
    with open(file_path, "r", encoding="utf-8") as f:
        records = json.load(f)

    df_res = pd.DataFrame(records)
    # Infer prompt_type if not carried
    prompt_type = df_res["prompt_type"].iloc[0] if "prompt_type" in df_res.columns else file_path.stem

    # Merge with ground truth on id (left join to keep only evaluated rows)
    key = "id"
    df_truth = df_truth.copy()
    if key not in df_truth.columns:
        raise ValueError("df_truth must include an 'id' column.")

    # Build tidy frame
    if task == "binary":
        if "label" not in df_truth.columns:
            raise ValueError("df_truth for binary must include a 'label' column with values in {'IPV','NOT_IPV'}")
        # y_true
        merged = df_res.merge(df_truth[["id", "label"]], on="id", how="inner")
        merged["y_true"] = merged["label"].map({"IPV": 1, "NOT_IPV": 0})
        # y_pred from extracted_label or response
        if "extracted_label" in merged.columns:
            pred_str = merged["extracted_label"]
        elif "response" in merged.columns:
            pred_str = merged["response"]
        else:
            raise ValueError("Binary results must contain 'extracted_label' or 'response'.")

        merged["y_pred"] = (pred_str.astype(str).str.upper() == "IPV").astype(int)

        # Optional scores
        scores = []
        has_scores = False
        for _, r in merged.iterrows():
            s = _try_extract_scores(r.to_dict())
            if s and any(k.upper() in ("IPV", "NOT_IPV") for k in s.keys()):
                has_scores = True
                # prefer IPV score if present; else invert NOT_IPV
                if "IPV" in s:
                    scores.append(float(s["IPV"]))
                elif "NOT_IPV" in s:
                    scores.append(1.0 - float(s["NOT_IPV"]))
                else:
                    scores.append(np.nan)
            else:
                scores.append(np.nan)
        merged["y_score"] = scores

        tidy = merged[["id", "y_true", "y_pred", "y_score"]].copy()

    elif task == "multilabel":
        # Ground truth expects boolean columns for each subtype
        need_cols = _one_vs_rest_columns()
        for c in need_cols:
            if c not in df_truth.columns:
                raise ValueError(f"df_truth for multilabel must include boolean column '{c}'")

        merged = df_res.merge(df_truth[["id"] + need_cols], on="id", how="inner")

        # Predictions: list of strings in 'extracted_labels' or 'response'
        if "extracted_labels" in merged.columns:
            preds = merged["extracted_labels"].apply(_ensure_list)
        elif "response" in merged.columns:
            preds = merged["response"].apply(_ensure_list)
        else:
            raise ValueError("Multilabel results must contain 'extracted_labels' or 'response' (list or comma string).")

        for c in need_cols:
            merged[f"y_true_{c}"] = merged[c].astype(int)
            merged[f"y_pred_{c}"] = preds.apply(lambda L: int(c in set(map(str, L))))

        # Optional per-class scores if present
        has_scores = False
        for c in need_cols:
            merged[f"y_score_{c}"] = np.nan

        for idx, r in merged.iterrows():
            s = _try_extract_scores(r.to_dict())
            if s:
                has_scores = True
                # normalize keys to capitalized class names if possible
                norm = {k.strip().capitalize(): v for k, v in s.items()}
                for c in need_cols:
                    if c in norm:
                        merged.at[idx, f"y_score_{c}"] = float(norm[c])

        tidy_cols = ["id"] + \
                    [f"y_true_{c}" for c in need_cols] + \
                    [f"y_pred_{c}" for c in need_cols] + \
                    [f"y_score_{c}" for c in need_cols]
        tidy = merged[tidy_cols].copy()

    else:
        raise ValueError("task must be 'binary' or 'multilabel'.")

    # Cache
    tidy.to_parquet(cache_key, index=False)
    meta = {"prompt_type": prompt_type, "has_scores": has_scores}
    (cache_dir / f"{file_path.stem}_{stamp}.meta.json").write_text(json.dumps(meta, indent=2))

    return {
        "file": str(file_path),
        "prompt_type": prompt_type,
        "task": task,
        "df": tidy.copy(),
        "has_scores": has_scores,
    }


    # ---------- Metrics -----------------------------------------------------------

def _binary_metrics(df_bin: pd.DataFrame) -> Dict[str, float]:
    from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
    y_true = df_bin["y_true"].values
    y_pred = df_bin["y_pred"].values.astype(int)
    acc = float(accuracy_score(y_true, y_pred))
    f1  = float(f1_score(y_true, y_pred, zero_division=0))
    # AUC if scores available
    if df_bin["y_score"].notna().any():
        y_score = df_bin["y_score"].fillna(0.5).values
        try:
            auc = float(roc_auc_score(y_true, y_score))
        except Exception:
            auc = float("nan")
    else:
        auc = float("nan")
    return {"Accuracy": acc, "F1": f1, "ROC_AUC": auc}

def _multilabel_metrics(df_ml: pd.DataFrame) -> Dict[str, float]:
    from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
    classes = _one_vs_rest_columns()

    # Macro F1 across the three abuse types (ignoring NOT_IPV)
    y_true = np.vstack([df_ml[f"y_true_{c}"].values for c in classes]).T
    y_pred = np.vstack([df_ml[f"y_pred_{c}"].values for c in classes]).T
    f1_macro = float(f1_score(y_true, y_pred, average="macro", zero_division=0))
    f1_micro = float(f1_score(y_true, y_pred, average="micro", zero_division=0))

    # "Exact match" accuracy: all three match
    exact_acc = float((y_true == y_pred).all(axis=1).mean())

    # Per-class AUC if scores present
    aucs = {}
    have_any_scores = False
    for c in classes:
        s = df_ml[f"y_score_{c}"]
        if s.notna().any():
            have_any_scores = True
            try:
                aucs[c] = float(roc_auc_score(df_ml[f"y_true_{c}"].values, s.fillna(0.5).values))
            except Exception:
                aucs[c] = float("nan")
        else:
            aucs[c] = float("nan")

    # Aggregate AUC (macro) across available classes
    if have_any_scores:
        auc_macro = float(np.nanmean([aucs[c] for c in classes]))
    else:
        auc_macro = float("nan")

    out = {
        "F1_macro": f1_macro,
        "F1_micro": f1_micro,
        "ExactMatchAcc": exact_acc,
        "ROC_AUC_macro": auc_macro,
    }
    # include per-class AUC columns
    for c in classes:
        out[f"AUC_{c}"] = aucs[c]
    return out

# ---------- Prompt comparison over a directory --------------------------------

def compare_prompts(
    results_dir: str | Path,
    df_truth_binary: pd.DataFrame,
    df_truth_multilabel: pd.DataFrame,
) -> Dict[str, pd.DataFrame]:
    """
    Loop test_results directory, process each file once (using cache),
    compute metrics, and return two DataFrames:
      - df_cmp_binary: metrics per binary prompt
      - df_cmp_multilabel: metrics per multilabel prompt
    Also returns best prompt names for each task.
    """
    results_dir = Path(results_dir)

    rows_bin, rows_ml = [], []

    for fp in sorted(results_dir.glob("*.json")):
        name = fp.stem
        if name.startswith("binary_"):
            info = process_results_file(fp, df_truth_binary, task="binary")
            metrics = _binary_metrics(info["df"])
            row = {"prompt": info["prompt_type"], **metrics, "file": str(fp)}
            rows_bin.append(row)

        elif name.startswith("multilabel_"):
            info = process_results_file(fp, df_truth_multilabel, task="multilabel")
            metrics = _multilabel_metrics(info["df"])
            row = {"prompt": info["prompt_type"], **metrics, "file": str(fp)}
            rows_ml.append(row)

    df_cmp_binary = pd.DataFrame(rows_bin).sort_values(["F1", "ROC_AUC", "Accuracy"], ascending=False)
    df_cmp_multilabel = pd.DataFrame(rows_ml).sort_values(["F1_macro", "ROC_AUC_macro", "ExactMatchAcc"], ascending=False)

    best_binary = df_cmp_binary.iloc[0]["prompt"] if not df_cmp_binary.empty else None
    best_multilabel = df_cmp_multilabel.iloc[0]["prompt"] if not df_cmp_multilabel.empty else None

    return {
        "binary": df_cmp_binary.reset_index(drop=True),
        "multilabel": df_cmp_multilabel.reset_index(drop=True),
        "best_binary": best_binary,
        "best_multilabel": best_multilabel,
    }

# ---------- Visualization (comparison phase) ----------------------------------

def plot_prompt_comparison(
    df_cmp_binary: pd.DataFrame,
    df_cmp_multilabel: pd.DataFrame,
    out_dir: str | Path = "../1_LLM_Eval/figs"
):
    """
    Create simple tables and barplots comparing prompts (AUC/F1).
    Uses matplotlib only.
    """
    import matplotlib.pyplot as plt
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Binary: bar plot F1 and AUC
    if not df_cmp_binary.empty:
        plt.figure()
        x = np.arange(len(df_cmp_binary))
        plt.bar(x - 0.2, df_cmp_binary["F1"], width=0.4, label="F1")
        # Safe AUC display
        auc_vals = df_cmp_binary["ROC_AUC"].fillna(0)
        plt.bar(x + 0.2, auc_vals, width=0.4, label="ROC-AUC")
        plt.xticks(x, df_cmp_binary["prompt"], rotation=45, ha="right")
        plt.ylabel("Score")
        plt.title("Binary Prompt Comparison")
        plt.legend()
        plt.tight_layout()
        plt.savefig(out_dir / "binary_prompt_comparison.png", dpi=150)
        plt.close()

    # Multilabel: bar plot Macro F1 and Macro AUC
    if not df_cmp_multilabel.empty:
        plt.figure()
        x = np.arange(len(df_cmp_multilabel))
        plt.bar(x - 0.2, df_cmp_multilabel["F1_macro"], width=0.4, label="F1_macro")
        auc_vals = df_cmp_multilabel["ROC_AUC_macro"].fillna(0)
        plt.bar(x + 0.2, auc_vals, width=0.4, label="ROC_AUC_macro")
        plt.xticks(x, df_cmp_multilabel["prompt"], rotation=45, ha="right")
        plt.ylabel("Score")
        plt.title("Multilabel Prompt Comparison")
        plt.legend()
        plt.tight_layout()
        plt.savefig(out_dir / "multilabel_prompt_comparison.png", dpi=150)
        plt.close()

# ---------- Final Evaluation with selected prompt (no reprocessing) -----------

def final_evaluation_binary(
    best_file_path: str | Path,
    df_truth_binary: pd.DataFrame
):
    """
    Use the cached, processed outputs for the best binary prompt to:
      - Compute ROC curve, PR curve, Accuracy, F1
      - Produce a waterfall plot (sorted by score if available; else by prediction)
    Saves figures to ../1_LLM_Eval/figs.
    """
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, precision_recall_curve, auc, accuracy_score, f1_score

    info = process_results_file(best_file_path, df_truth_binary, task="binary")
    dfb = info["df"]
    out_dir = Path("../1_LLM_Eval/figs"); out_dir.mkdir(parents=True, exist_ok=True)

    y_true = dfb["y_true"].values
    y_pred = dfb["y_pred"].values
    acc = float(accuracy_score(y_true, y_pred))
    f1  = float(f1_score(y_true, y_pred, zero_division=0))

    # ROC + PR if scores available; otherwise use 0/1 "scores"
    if dfb["y_score"].notna().any():
        scores = dfb["y_score"].fillna(0.5).values
    else:
        scores = y_pred.astype(float)

    fpr, tpr, _ = roc_curve(y_true, scores)
    roc_auc = float(auc(fpr, tpr))

    prec, rec, _ = precision_recall_curve(y_true, scores)
    pr_auc = float(auc(rec, prec))

    # Save ROC
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
    plt.plot([0,1], [0,1], linestyle="--")
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("Binary ROC")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / "binary_ROC.png", dpi=150); plt.close()

    # Save PR
    plt.figure()
    plt.plot(rec, prec, label=f"PR AUC={pr_auc:.3f}")
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Binary PR Curve")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / "binary_PR.png", dpi=150); plt.close()

    # Waterfall: sort by score desc
    order = np.argsort(-scores)
    plt.figure()
    plt.bar(np.arange(len(scores)), scores[order], edgecolor="none")
    # overlay color by true class (blue=NOT_IPV 0, orange=IPV 1)
    for idx, o in enumerate(order):
        if y_true[o] == 1:
            plt.plot(idx, scores[order][idx], marker="o")  # simple overlay
    plt.xlabel("Samples (sorted by score)"); plt.ylabel("Score")
    plt.title("Binary Waterfall (dots mark true IPV=1)")
    plt.tight_layout()
    plt.savefig(out_dir / "binary_waterfall.png", dpi=150); plt.close()

    return {"Accuracy": acc, "F1": f1, "ROC_AUC": roc_auc, "PR_AUC": pr_auc}

def final_evaluation_multilabel(
    best_file_path: str | Path,
    df_truth_multilabel: pd.DataFrame
):
    """
    Use the cached, processed outputs for the best multi-label prompt to:
      - Compute per-type ROC/AUC (Physical, Emotional, Sexual)
      - Bar chart comparing type-specific AUCs and Macro F1
      - Waterfall plot for each subtype (sorted by score if available)
    Saves figures to ../1_LLM_Eval/figs.
    """
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc, f1_score

    classes = _one_vs_rest_columns()
    info = process_results_file(best_file_path, df_truth_multilabel, task="multilabel")
    dfm = info["df"]
    out_dir = Path("../1_LLM_Eval/figs"); out_dir.mkdir(parents=True, exist_ok=True)

    # Metrics
    f1_macro = float(f1_score(
        np.vstack([dfm[f"y_true_{c}"] for c in classes]).T,
        np.vstack([dfm[f"y_pred_{c}"] for c in classes]).T,
        average="macro",
        zero_division=0
    ))

    aucs = {}
    for c in classes:
        y_true = dfm[f"y_true_{c}"].values
        s = dfm[f"y_score_{c}"]
        if s.notna().any():
            scores = s.fillna(0.5).values
        else:
            scores = dfm[f"y_pred_{c}"].values.astype(float)  # degenerate but allows plotting

        fpr, tpr, _ = roc_curve(y_true, scores)
        aucs[c] = float(auc(fpr, tpr))

        # Save subtype ROC
        plt.figure()
        plt.plot(fpr, tpr, label=f"AUC={aucs[c]:.3f}")
        plt.plot([0,1], [0,1], linestyle="--")
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {c}")
        plt.legend(); plt.tight_layout()
        plt.savefig(out_dir / f"multilabel_ROC_{c}.png", dpi=150); plt.close()

        # Waterfall per subtype
        order = np.argsort(-scores)
        plt.figure()
        plt.bar(np.arange(len(scores)), scores[order], edgecolor="none")
        for idx, o in enumerate(order):
            if y_true[o] == 1:
                plt.plot(idx, scores[order][idx], marker="o")
        plt.xlabel("Samples (sorted by score)"); plt.ylabel("Score")
        plt.title(f"Waterfall — {c} (dots mark true {c}=1)")
        plt.tight_layout()
        plt.savefig(out_dir / f"multilabel_waterfall_{c}.png", dpi=150); plt.close()

    # Summary bar: AUCs
    plt.figure()
    xs = np.arange(len(classes))
    plt.bar(xs, [aucs[c] for c in classes])
    plt.xticks(xs, classes)
    plt.ylabel("AUC")
    plt.title(f"Type-specific AUCs (Macro F1={f1_macro:.3f})")
    plt.tight_layout()
    plt.savefig(out_dir / "multilabel_type_AUCs.png", dpi=150); plt.close()

    return {"F1_macro": f1_macro, **{f"AUC_{c}": aucs[c] for c in classes}}

# ---------- Example driver (glue) ---------------------------------------------

def run_prompt_comparison_and_final(
    results_dir="../1_LLM_Eval/test_results",
    df_truth_binary: pd.DataFrame = None,
    df_truth_multilabel: pd.DataFrame = None,
):
    """
    1) Prompt Comparison Phase: compute metrics per prompt, plot comparisons,
       identify best prompts for binary and multilabel.
    2) Final Evaluation with the best prompt (uses cached processing).
    Returns a dict with DataFrames, best names, and final metrics.
    """
    cmp_out = compare_prompts(results_dir, df_truth_binary, df_truth_multilabel)
    plot_prompt_comparison(cmp_out["binary"], cmp_out["multilabel"])

    # Find the files for best prompts without reprocessing
    res_dir = Path(results_dir)
    best_bin_name = cmp_out["best_binary"]
    best_ml_name  = cmp_out["best_multilabel"]

    best_bin_file = None
    best_ml_file  = None
    if best_bin_name:
        # pick the latest file that matches binary_bestBinName_*.json
        candidates = sorted(res_dir.glob(f"binary_{best_bin_name}_*.json"))
        if candidates:
            best_bin_file = candidates[-1]
    if best_ml_name:
        candidates = sorted(res_dir.glob(f"multilabel_{best_ml_name}_*.json"))
        if candidates:
            best_ml_file = candidates[-1]

    final_bin = final_ml = None
    if best_bin_file:
        final_bin = final_evaluation_binary(best_bin_file, df_truth_binary)
    if best_ml_file:
        final_ml  = final_evaluation_multilabel(best_ml_file, df_truth_multilabel)

    return {
        "comparison_binary": cmp_out["binary"],
        "comparison_multilabel": cmp_out["multilabel"],
        "best_binary": best_bin_name,
        "best_multilabel": best_ml_name,
        "final_binary_metrics": final_bin,
        "final_multilabel_metrics": final_ml,
    }


In [2]:
# ---------- Metrics -----------------------------------------------------------

def _binary_metrics(df_bin: pd.DataFrame) -> Dict[str, float]:
    from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
    y_true = df_bin["y_true"].values
    y_pred = df_bin["y_pred"].values.astype(int)
    acc = float(accuracy_score(y_true, y_pred))
    f1  = float(f1_score(y_true, y_pred, zero_division=0))
    # AUC if scores available
    if df_bin["y_score"].notna().any():
        y_score = df_bin["y_score"].fillna(0.5).values
        try:
            auc = float(roc_auc_score(y_true, y_score))
        except Exception:
            auc = float("nan")
    else:
        auc = float("nan")
    return {"Accuracy": acc, "F1": f1, "ROC_AUC": auc}

def _multilabel_metrics(df_ml: pd.DataFrame) -> Dict[str, float]:
    from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
    classes = _one_vs_rest_columns()

    # Macro F1 across the three abuse types (ignoring NOT_IPV)
    y_true = np.vstack([df_ml[f"y_true_{c}"].values for c in classes]).T
    y_pred = np.vstack([df_ml[f"y_pred_{c}"].values for c in classes]).T
    f1_macro = float(f1_score(y_true, y_pred, average="macro", zero_division=0))
    f1_micro = float(f1_score(y_true, y_pred, average="micro", zero_division=0))

    # "Exact match" accuracy: all three match
    exact_acc = float((y_true == y_pred).all(axis=1).mean())

    # Per-class AUC if scores present
    aucs = {}
    have_any_scores = False
    for c in classes:
        s = df_ml[f"y_score_{c}"]
        if s.notna().any():
            have_any_scores = True
            try:
                aucs[c] = float(roc_auc_score(df_ml[f"y_true_{c}"].values, s.fillna(0.5).values))
            except Exception:
                aucs[c] = float("nan")
        else:
            aucs[c] = float("nan")

    # Aggregate AUC (macro) across available classes
    if have_any_scores:
        auc_macro = float(np.nanmean([aucs[c] for c in classes]))
    else:
        auc_macro = float("nan")

    out = {
        "F1_macro": f1_macro,
        "F1_micro": f1_micro,
        "ExactMatchAcc": exact_acc,
        "ROC_AUC_macro": auc_macro,
    }
    # include per-class AUC columns
    for c in classes:
        out[f"AUC_{c}"] = aucs[c]
    return out

# ---------- Prompt comparison over a directory --------------------------------

def compare_prompts(
    results_dir: str | Path,
    df_truth_binary: pd.DataFrame,
    df_truth_multilabel: pd.DataFrame,
) -> Dict[str, pd.DataFrame]:
    """
    Loop test_results directory, process each file once (using cache),
    compute metrics, and return two DataFrames:
      - df_cmp_binary: metrics per binary prompt
      - df_cmp_multilabel: metrics per multilabel prompt
    Also returns best prompt names for each task.
    """
    results_dir = Path(results_dir)

    rows_bin, rows_ml = [], []

    for fp in sorted(results_dir.glob("*.json")):
        name = fp.stem
        if name.startswith("binary_"):
            info = process_results_file(fp, df_truth_binary, task="binary")
            metrics = _binary_metrics(info["df"])
            row = {"prompt": info["prompt_type"], **metrics, "file": str(fp)}
            rows_bin.append(row)

        elif name.startswith("multilabel_"):
            info = process_results_file(fp, df_truth_multilabel, task="multilabel")
            metrics = _multilabel_metrics(info["df"])
            row = {"prompt": info["prompt_type"], **metrics, "file": str(fp)}
            rows_ml.append(row)

    df_cmp_binary = pd.DataFrame(rows_bin).sort_values(["F1", "ROC_AUC", "Accuracy"], ascending=False)
    df_cmp_multilabel = pd.DataFrame(rows_ml).sort_values(["F1_macro", "ROC_AUC_macro", "ExactMatchAcc"], ascending=False)

    best_binary = df_cmp_binary.iloc[0]["prompt"] if not df_cmp_binary.empty else None
    best_multilabel = df_cmp_multilabel.iloc[0]["prompt"] if not df_cmp_multilabel.empty else None

    return {
        "binary": df_cmp_binary.reset_index(drop=True),
        "multilabel": df_cmp_multilabel.reset_index(drop=True),
        "best_binary": best_binary,
        "best_multilabel": best_multilabel,
    }

# ---------- Visualization (comparison phase) ----------------------------------

def plot_prompt_comparison(
    df_cmp_binary: pd.DataFrame,
    df_cmp_multilabel: pd.DataFrame,
    out_dir: str | Path = "../1_LLM_Eval/figs"
):
    """
    Create simple tables and barplots comparing prompts (AUC/F1).
    Uses matplotlib only.
    """
    import matplotlib.pyplot as plt
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Binary: bar plot F1 and AUC
    if not df_cmp_binary.empty:
        plt.figure()
        x = np.arange(len(df_cmp_binary))
        plt.bar(x - 0.2, df_cmp_binary["F1"], width=0.4, label="F1")
        # Safe AUC display
        auc_vals = df_cmp_binary["ROC_AUC"].fillna(0)
        plt.bar(x + 0.2, auc_vals, width=0.4, label="ROC-AUC")
        plt.xticks(x, df_cmp_binary["prompt"], rotation=45, ha="right")
        plt.ylabel("Score")
        plt.title("Binary Prompt Comparison")
        plt.legend()
        plt.tight_layout()
        plt.savefig(out_dir / "binary_prompt_comparison.png", dpi=150)
        plt.close()

    # Multilabel: bar plot Macro F1 and Macro AUC
    if not df_cmp_multilabel.empty:
        plt.figure()
        x = np.arange(len(df_cmp_multilabel))
        plt.bar(x - 0.2, df_cmp_multilabel["F1_macro"], width=0.4, label="F1_macro")
        auc_vals = df_cmp_multilabel["ROC_AUC_macro"].fillna(0)
        plt.bar(x + 0.2, auc_vals, width=0.4, label="ROC_AUC_macro")
        plt.xticks(x, df_cmp_multilabel["prompt"], rotation=45, ha="right")
        plt.ylabel("Score")
        plt.title("Multilabel Prompt Comparison")
        plt.legend()
        plt.tight_layout()
        plt.savefig(out_dir / "multilabel_prompt_comparison.png", dpi=150)
        plt.close()

# ---------- Final Evaluation with selected prompt (no reprocessing) -----------

def final_evaluation_binary(
    best_file_path: str | Path,
    df_truth_binary: pd.DataFrame
):
    """
    Use the cached, processed outputs for the best binary prompt to:
      - Compute ROC curve, PR curve, Accuracy, F1
      - Produce a waterfall plot (sorted by score if available; else by prediction)
    Saves figures to ../1_LLM_Eval/figs.
    """
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, precision_recall_curve, auc, accuracy_score, f1_score

    info = process_results_file(best_file_path, df_truth_binary, task="binary")
    dfb = info["df"]
    out_dir = Path("../1_LLM_Eval/figs"); out_dir.mkdir(parents=True, exist_ok=True)

    y_true = dfb["y_true"].values
    y_pred = dfb["y_pred"].values
    acc = float(accuracy_score(y_true, y_pred))
    f1  = float(f1_score(y_true, y_pred, zero_division=0))

    # ROC + PR if scores available; otherwise use 0/1 "scores"
    if dfb["y_score"].notna().any():
        scores = dfb["y_score"].fillna(0.5).values
    else:
        scores = y_pred.astype(float)

    fpr, tpr, _ = roc_curve(y_true, scores)
    roc_auc = float(auc(fpr, tpr))

    prec, rec, _ = precision_recall_curve(y_true, scores)
    pr_auc = float(auc(rec, prec))

    # Save ROC
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
    plt.plot([0,1], [0,1], linestyle="--")
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("Binary ROC")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / "binary_ROC.png", dpi=150); plt.close()

    # Save PR
    plt.figure()
    plt.plot(rec, prec, label=f"PR AUC={pr_auc:.3f}")
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Binary PR Curve")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / "binary_PR.png", dpi=150); plt.close()

    # Waterfall: sort by score desc
    order = np.argsort(-scores)
    plt.figure()
    plt.bar(np.arange(len(scores)), scores[order], edgecolor="none")
    # overlay color by true class (blue=NOT_IPV 0, orange=IPV 1)
    for idx, o in enumerate(order):
        if y_true[o] == 1:
            plt.plot(idx, scores[order][idx], marker="o")  # simple overlay
    plt.xlabel("Samples (sorted by score)"); plt.ylabel("Score")
    plt.title("Binary Waterfall (dots mark true IPV=1)")
    plt.tight_layout()
    plt.savefig(out_dir / "binary_waterfall.png", dpi=150); plt.close()

    return {"Accuracy": acc, "F1": f1, "ROC_AUC": roc_auc, "PR_AUC": pr_auc}

def final_evaluation_multilabel(
    best_file_path: str | Path,
    df_truth_multilabel: pd.DataFrame
):
    """
    Use the cached, processed outputs for the best multi-label prompt to:
      - Compute per-type ROC/AUC (Physical, Emotional, Sexual)
      - Bar chart comparing type-specific AUCs and Macro F1
      - Waterfall plot for each subtype (sorted by score if available)
    Saves figures to ../1_LLM_Eval/figs.
    """
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc, f1_score

    classes = _one_vs_rest_columns()
    info = process_results_file(best_file_path, df_truth_multilabel, task="multilabel")
    dfm = info["df"]
    out_dir = Path("../1_LLM_Eval/figs"); out_dir.mkdir(parents=True, exist_ok=True)

    # Metrics
    f1_macro = float(f1_score(
        np.vstack([dfm[f"y_true_{c}"] for c in classes]).T,
        np.vstack([dfm[f"y_pred_{c}"] for c in classes]).T,
        average="macro",
        zero_division=0
    ))

    aucs = {}
    for c in classes:
        y_true = dfm[f"y_true_{c}"].values
        s = dfm[f"y_score_{c}"]
        if s.notna().any():
            scores = s.fillna(0.5).values
        else:
            scores = dfm[f"y_pred_{c}"].values.astype(float)  # degenerate but allows plotting

        fpr, tpr, _ = roc_curve(y_true, scores)
        aucs[c] = float(auc(fpr, tpr))

        # Save subtype ROC
        plt.figure()
        plt.plot(fpr, tpr, label=f"AUC={aucs[c]:.3f}")
        plt.plot([0,1], [0,1], linestyle="--")
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {c}")
        plt.legend(); plt.tight_layout()
        plt.savefig(out_dir / f"multilabel_ROC_{c}.png", dpi=150); plt.close()

        # Waterfall per subtype
        order = np.argsort(-scores)
        plt.figure()
        plt.bar(np.arange(len(scores)), scores[order], edgecolor="none")
        for idx, o in enumerate(order):
            if y_true[o] == 1:
                plt.plot(idx, scores[order][idx], marker="o")
        plt.xlabel("Samples (sorted by score)"); plt.ylabel("Score")
        plt.title(f"Waterfall — {c} (dots mark true {c}=1)")
        plt.tight_layout()
        plt.savefig(out_dir / f"multilabel_waterfall_{c}.png", dpi=150); plt.close()

    # Summary bar: AUCs
    plt.figure()
    xs = np.arange(len(classes))
    plt.bar(xs, [aucs[c] for c in classes])
    plt.xticks(xs, classes)
    plt.ylabel("AUC")
    plt.title(f"Type-specific AUCs (Macro F1={f1_macro:.3f})")
    plt.tight_layout()
    plt.savefig(out_dir / "multilabel_type_AUCs.png", dpi=150); plt.close()

    return {"F1_macro": f1_macro, **{f"AUC_{c}": aucs[c] for c in classes}}

# ---------- Example driver (glue) ---------------------------------------------

def run_prompt_comparison_and_final(
    results_dir="../1_LLM_Eval/test_results",
    df_truth_binary: pd.DataFrame = None,
    df_truth_multilabel: pd.DataFrame = None,
):
    """
    1) Prompt Comparison Phase: compute metrics per prompt, plot comparisons,
       identify best prompts for binary and multilabel.
    2) Final Evaluation with the best prompt (uses cached processing).
    Returns a dict with DataFrames, best names, and final metrics.
    """
    cmp_out = compare_prompts(results_dir, df_truth_binary, df_truth_multilabel)
    plot_prompt_comparison(cmp_out["binary"], cmp_out["multilabel"])

    # Find the files for best prompts without reprocessing
    res_dir = Path(results_dir)
    best_bin_name = cmp_out["best_binary"]
    best_ml_name  = cmp_out["best_multilabel"]

    best_bin_file = None
    best_ml_file  = None
    if best_bin_name:
        # pick the latest file that matches binary_bestBinName_*.json
        candidates = sorted(res_dir.glob(f"binary_{best_bin_name}_*.json"))
        if candidates:
            best_bin_file = candidates[-1]
    if best_ml_name:
        candidates = sorted(res_dir.glob(f"multilabel_{best_ml_name}_*.json"))
        if candidates:
            best_ml_file = candidates[-1]

    final_bin = final_ml = None
    if best_bin_file:
        final_bin = final_evaluation_binary(best_bin_file, df_truth_binary)
    if best_ml_file:
        final_ml  = final_evaluation_multilabel(best_ml_file, df_truth_multilabel)

    return {
        "comparison_binary": cmp_out["binary"],
        "comparison_multilabel": cmp_out["multilabel"],
        "best_binary": best_bin_name,
        "best_multilabel": best_ml_name,
        "final_binary_metrics": final_bin,
        "final_multilabel_metrics": final_ml,
    }


# Actual Testing

**Loading Ground Truth CSV**

In [4]:
# Load your full dataset
df = pd.read_csv("/content/drive/MyDrive/ECs & Work/Research/617points.csv")

# Ensure there's an 'id' column (if not, create one from index)
if "id" not in df.columns:
    df["id"] = df.index

# ---------------------------
# 1️⃣ Binary ground truth
# ---------------------------
# Define IPV = if ANY of the 3 abuse labels is True
df["IPV"] = df[["Physical Abuse", "Emotional Abuse", "Sexual Abuse"]].any(axis=1)

df_truth_binary = df[["id", "IPV"]].copy()
df_truth_binary["label"] = df_truth_binary["IPV"].map({True: "IPV", False: "NOT_IPV"})
df_truth_binary = df_truth_binary[["id", "label"]]  # reorder columns

# ---------------------------
# 2️⃣ Multilabel ground truth
# ---------------------------
df_truth_multilabel = df.rename(
    columns={
        "Physical Abuse": "Physical",
        "Emotional Abuse": "Emotional",
        "Sexual Abuse": "Sexual"
    }
)[["id", "Physical", "Emotional", "Sexual"]].copy()

# Convert to 0/1
df_truth_multilabel = df_truth_multilabel.astype({
    "Physical": int,
    "Emotional": int,
    "Sexual": int
})

In [6]:
out = run_prompt_comparison_and_final(
    results_dir="/content/drive/MyDrive/ECs/Research",
    df_truth_binary=df_truth_binary,
    df_truth_multilabel=df_truth_multilabel
)
print(out["comparison_binary"])
print(out["comparison_multilabel"])
print("Best Binary Prompt:", out["best_binary"])
print("Best Multilabel Prompt:", out["best_multilabel"])
print("Final Binary Metrics:", out["final_binary_metrics"])
print("Final Multilabel Metrics:", out["final_multilabel_metrics"])


KeyError: 'F1_macro'